# Latencia
Tiempo que tarda un modelo en procesar una entrada y generar una prediccion

In [1]:
import os, time
import importlib , latency, pandas as pd # Importar importlib para recargar módulos
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), '..', '.env')
importlib.reload(latency)
from glob import glob
from latency import Latency

# Configuracion
config_file = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), os.getenv('CONFIG_SUBPATH'))
checkpoint_file = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), os.getenv('CHECKPOINT_SUBPATH'))
images_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('IMAGES_SUBPATH'))
labels_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('LABELS_SUBPATH'))
videos_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('VIDEOS_SUBPATH'))
evaluator = Latency(os.getenv('BASE_PATH'), images_path, labels_path, videos_path, config_file, checkpoint_file)
thresholds = [0.1, 0.5, 0.9]

/Users/apple/Documents/pose_estimation/MMPosev1.1/mmpose_env/lib/python3.10/site-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv`` rather than ``mmcv-lite`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


Loads checkpoint by local backend from path: /Users/apple/Documents/pose_estimation/MMPosev1.1/configs/model_weights/hrnet_w32_coco_256x192-c78dce93_20200708.pth


/Users/apple/Documents/pose_estimation/MMPosev1.1/mmpose_env/lib/python3.10/site-packages/mmpose/datasets/datasets/utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/coco.py" does not exist. A matched config file "/Users/apple/Documents/pose_estimation/MMPosev1.1/mmpose_env/lib/python3.10/site-packages/mmpose/.mim/configs/_base_/datasets/coco.py" will be used instead.
  warnings.warn(


# Calculo de latencia en fotografias (1 imagen)


In [2]:
image = 'imagen_014_jpg.rf.9b73844fd1a58e6de1dd2cd81aca8dc6.jpg'
results = []
for threshold in thresholds:
    results = evaluator.evaluate_image(image, threshold, results)

# Calculo de latencia en fotografias (muchas)

In [2]:
# Obtener la lista de todas las imágenes en la carpeta
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
image_files_sorted = sorted(image_files)
results = []

for image in image_files_sorted:
    for threshold in thresholds:
        results = evaluator.evaluate_image(image, threshold, results)

# Generando XLS

In [3]:
df_results = pd.DataFrame(results)
print(df_results.head().to_string)
output_excel_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), 'velocidad', 'latency', 'mmpose_latency_results.xlsx')
df_results.to_excel(output_excel_path, index=False, engine='openpyxl')

print(f"Los resultados se han guardado en {output_excel_path}")

<bound method DataFrame.to_string of                                                image image_size  threshold  \
0  imagen_002_jpg.rf.39766b197a04d8d1569b3e546fd0...    144x256        0.1   
1  imagen_002_jpg.rf.39766b197a04d8d1569b3e546fd0...    144x256        0.5   
2  imagen_002_jpg.rf.39766b197a04d8d1569b3e546fd0...    144x256        0.9   
3  imagen_003_jpg.rf.212077a70b1de84df04262b03db5...    147x256        0.1   
4  imagen_003_jpg.rf.212077a70b1de84df04262b03db5...    147x256        0.5   

      latency  
0  268.116951  
1  232.179880  
2  234.758139  
3  234.201193  
4  249.929905  >
Los resultados se han guardado en /Users/apple/Documents/pose_estimation/MMPosev1.1/velocidad/latency/mmpose_latency_results.xlsx


# Calculo de latencia en un video

In [5]:
video = 'ejercicio01 - frente.mp4'
video_path = os.path.join(videos_path, 'uziel', 'ejercicio01', 'conLuz', video)
results = []

for threshold in thresholds:
    results = evaluator.evaluate_video(
        videos_path=video_path, 
        threshold=threshold, 
        results=results,
        frame_skip=5,
    )

Procesando ejercicio01 - frente.mp4: 100%|██████████| 175/175 [00:08<00:00, 19.95it/s]


# Calculo de latencia en videos dentro de una carpeta

In [ ]:

folder_path = os.path.join(videos_path, 'uziel')
video_extensions = ['*.mp4', '*.avi', '*.mov']
video_files = []
results = []

for ext in video_extensions:
    video_files.extend(glob(os.path.join(folder_path, '**', ext), recursive=True))

for video_path in video_files:
    for threshold in thresholds:
        results = evaluator.evaluate_video(
            videos_path=video_path, 
            threshold=threshold, 
            results=results,
            frame_skip=5,
        )

# Generando XLS de los resultados de los videos

In [6]:
df_results = pd.DataFrame(results)
print(df_results.head().to_string)
output_excel_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), 'latency', 'video_latency_results.xlsx')
df_results.to_excel(output_excel_path, index=False, engine='openpyxl')

print(f"Los resultados se han guardado en {output_excel_path}")

<bound method DataFrame.to_string of                       video video_resolution  threshold  video_fps  \
0  ejercicio01 - frente.mp4          480x864        0.1  30.016466   
1  ejercicio01 - frente.mp4          480x864        0.1  30.016466   
2  ejercicio01 - frente.mp4          480x864        0.1  30.016466   
3  ejercicio01 - frente.mp4          480x864        0.1  30.016466   
4  ejercicio01 - frente.mp4          480x864        0.1  30.016466   

   frame_skip  latency_ms  processed_frames  total_frames  
0           5  250.088930                 1           175  
1           5  239.281178                 2           175  
2           5  251.007080                 3           175  
3           5  288.485765                 4           175  
4           5  263.056993                 5           175  >
Los resultados se han guardado en /Users/apple/Documents/uziel/MMPosev1.1/latency/video_latency_results.xlsx
